In [73]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import tifffile
import os
import matplotlib.pyplot as plt

In [74]:
# Read and stitch the image
folder_path = 'Result TIF'
file_list = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and filename != '.DS_Store':
        file_list.append(file_path)
# image = tifffile.imread('Result TIFF/h2114153 h&e.tif - Series 0_ROI_1.tif', aszarr=False)

### Generalization of Kenny's Code

In [67]:
def image_processing(file_name):
    image = tifffile.imread(file_name, aszarr = False)
    
    # Convert to LAB color space
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Extract the L channel
    l_channel = image_lab[:,:,0]

    # Apply Otsu's thresholding
    _, otsu_threshold = cv2.threshold(l_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Create a mask using the Otsu threshold
    mask = np.zeros(l_channel.shape, dtype=np.uint8)
    mask[l_channel > otsu_threshold] = 255

    # Ensure mask is the same size as the image
    if mask.shape[:2] != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Close any "holes" (the whitespace)
    kernel_size = max(1, min(image_lab.shape[0], image_lab.shape[1]) // 110)
    kernel = np.ones(kernel_size, np.uint8)
    closed_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(closed_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Remove small contours (get rid of the little spots in the middle)
    min_contour_area = (image_lab.shape[0] * image_lab.shape[1]) * 0.5
    # Create a directory to save the individual regions
    output_dir = 'purple_regions'
    os.makedirs(output_dir, exist_ok=True)

    # Process each large contour individually
    region_count = 0
    for contour in contours:
        if cv2.contourArea(contour) > min_contour_area:
            # Create a mask for the current contour
            contour_mask = np.zeros(image.shape[:2], dtype=np.uint8)
            cv2.drawContours(contour_mask, [contour], -1, 255, thickness=cv2.FILLED)

            # Apply the mask to the original image
            region = cv2.bitwise_and(image, image, mask=contour_mask)

            # Crop the region to the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(contour)
            cropped_region = region[y:y+h, x:x+w]

            # Save the cropped region as a TIFF image
            region_filename = f'{file[11:]}'
            region_path = os.path.join(output_dir, region_filename)
            tifffile.imwrite(region_path, cv2.cvtColor(cropped_region, cv2.COLOR_BGR2RGB))

        region_count += 1

In [68]:
for file in file_list:
    image_processing(file)

Saved h2114164 h&e.tif - Series 0_ROI_2.tif
Saved h2114164 h&e.tif - Series 0_ROI_4.tif
Saved h2114153 h&e.tif - Series 0_ROI_4.tif
Saved h2114153 h&e.tif - Series 0_ROI_1.tif
Saved h2114153 h&e.tif - Series 0_ROI_3.tif
Saved h2114166 h&e.tif - Series 0_ROI_2.tif
Saved h2114165 h&e.tif - Series 0_ROI_4.tif
Saved h2114166 h&e.tif - Series 0_ROI_5.tif


### Epithelium Separation using Otsu Threshold

In [221]:
def image_processing(file_name):
    image = tifffile.imread(file_name, aszarr = False)
    
    # Convert to LAB color space
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Extract the L channel
    l_channel = image_lab[:,:,0]

    # Apply Otsu's thresholding
    _, otsu_threshold = cv2.threshold(l_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Create a mask using the Otsu threshold
    mask = np.zeros(l_channel.shape, dtype=np.uint8)
    mask[l_channel > otsu_threshold] = 255

    # Ensure mask is the same size as the image
    if mask.shape[:2] != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    # Create a directory to save the processed image
    output_dir = 'processed_images'
    os.makedirs(output_dir, exist_ok=True)

    # Apply the mask to the original image
    processed_image = cv2.bitwise_and(image, image, mask=mask)

    # Save the processed image as a TIFF file
    output_filename = f'processed_{os.path.basename(file_name)}'
    output_path = os.path.join(output_dir, output_filename)
    tifffile.imwrite(output_path, cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB))

In [222]:
for file in file_list:
    image_processing(file)

### Eli's Sample Selection

In [242]:
# Directories
input_folder = "Result TIF"
output_folder = "filtered_images"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply a Gaussian blur to smooth the image (reduce blur kernel size)
    blurred_image = cv2.GaussianBlur(gray_image, (25, 25), 0)

    # Thresholding to segment the epithelium
    _, thresholded_mask = cv2.threshold(blurred_image, 150, 255, cv2.THRESH_BINARY_INV)

    # Morphological operations to clean up the mask
    kernel_small = np.ones((20, 20), np.uint8)
    cleaned_mask = cv2.morphologyEx(thresholded_mask, cv2.MORPH_CLOSE, kernel_small)
    cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_OPEN, kernel_small)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(cleaned_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(cleaned_mask)
        cv2.drawContours(epithelium_mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)
#         print(f"Average epithelium width for {image_path}: {average_epithelium_width}")

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

#         print(f"Processed and saved {output_image_path}, mask saved at {output_mask_path}")

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")
    
#     # Plotting multiple masks
#     fig, axes = plt.subplots(1, 4, figsize=(12, 6))

#     # First mask
#     axes[0].imshow(image, cmap='gray')
#     axes[0].set_title("Image")
#     axes[0].axis('off')
    
#     axes[1].imshow(blurred_image, cmap='gray')
#     axes[1].set_title("Blurred Image")
#     axes[1].axis('off')

#     # Second mask
#     axes[2].imshow(cleaned_mask, cmap='gray')
#     axes[2].set_title("Cleaned Mask")
#     axes[2].axis('off')
    
#     # Third mask
#     axes[3].imshow(epithelium_mask, cmap='gray')
#     axes[3].set_title("Epithelium Mask")
#     axes[3].axis('off')

#     # Show the plot
#     plt.tight_layout()
#     plt.show()

In [ ]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")

### Eli's Code Generalized

In [255]:
# Directories
input_folder = "processed_images"
output_folder = "Eli Generalized"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Thresholding to segment the epithelium
    _, thresholded_mask = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY_INV)

    # Morphological operations to clean up the mask
    kernel_size = max(1, min(gray_image.shape[0], gray_image.shape[1]) // 100)
    kernel_small = np.ones(kernel_size, np.uint8)
    cleaned_mask = cv2.morphologyEx(thresholded_mask, cv2.MORPH_CLOSE, kernel_small)
    cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_OPEN, kernel_small)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(cleaned_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(cleaned_mask)
        cv2.drawContours(epithelium_mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")

In [256]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")

Processing complete for all images.


### Eli's Code with just Otsu's

In [257]:
# Directories
input_folder = "Result TIF"
output_folder = "Eli + Otsu"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Thresholding to segment the epithelium
    _, thresholded_mask = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(thresholded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(thresholded_mask)
        cv2.drawContours(epithelium_mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)
#         print(f"Average epithelium width for {image_path}: {average_epithelium_width}")

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

#         print(f"Processed and saved {output_image_path}, mask saved at {output_mask_path}")

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")
    
#     # Plotting multiple masks
#     fig, axes = plt.subplots(1, 4, figsize=(12, 6))

#     # First mask
#     axes[0].imshow(image, cmap='gray')
#     axes[0].set_title("Image")
#     axes[0].axis('off')
    
#     axes[1].imshow(blurred_image, cmap='gray')
#     axes[1].set_title("Blurred Image")
#     axes[1].axis('off')

#     # Second mask
#     axes[2].imshow(thresholded_mask, cmap='gray')
#     axes[2].set_title("Cleaned Mask")
#     axes[2].axis('off')
    
#     # Third mask
#     axes[3].imshow(epithelium_mask, cmap='gray')
#     axes[3].set_title("Epithelium Mask")
#     axes[3].axis('off')

#     # Show the plot
#     plt.tight_layout()
#     plt.show()

In [258]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")

Processing complete for all images.


### Eli's Code with Otsu's on Grayscale + Contour Selection stopping at 10%

In [259]:
# Directories
input_folder = "Result TIF"
output_folder = "Eli + Otsu + 10%"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Thresholding to segment the epithelium
    _, thresholded_mask = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(thresholded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # Calculate areas for all contours and sort them in descending order
        contour_areas = [(cv2.contourArea(contour), contour) for contour in contours]
        contour_areas = sorted(contour_areas, key=lambda x: x[0], reverse=True)  # Sort based on area (first element of tuple)

        # Calculate the percentage difference between consecutive areas
        area_differences = []
        for i in range(len(contour_areas)-1):
            current_area = contour_areas[i][0]
            next_area = contour_areas[i+1][0]
            if current_area > 0:  # Avoid division by zero
                percentage = (next_area / current_area) * 100
                area_differences.append((percentage, i))
    
        # Find the index where there's a significant drop (e.g., >50% difference)
        cutoff_index = 0  # Default to just the largest contour
        for diff, idx in area_differences:
            if diff < 10:  # You can adjust this threshold
                cutoff_index = idx
                break

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(thresholded_mask)

        # Draw all contours up to the cutoff point
        selected_contours = [cont[1] for cont in contour_areas[:cutoff_index + 1]]
        cv2.drawContours(epithelium_mask, selected_contours, -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)
#         print(f"Average epithelium width for {image_path}: {average_epithelium_width}")

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

#         print(f"Processed and saved {output_image_path}, mask saved at {output_mask_path}")

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")
    
#     # Plotting multiple masks
#     fig, axes = plt.subplots(1, 4, figsize=(12, 6))

#     # First mask
#     axes[0].imshow(image, cmap='gray')
#     axes[0].set_title("Image")
#     axes[0].axis('off')
    
#     axes[1].imshow(blurred_image, cmap='gray')
#     axes[1].set_title("Blurred Image")
#     axes[1].axis('off')

#     # Second mask
#     axes[2].imshow(thresholded_mask, cmap='gray')
#     axes[2].set_title("Cleaned Mask")
#     axes[2].axis('off')
    
#     # Third mask
#     axes[3].imshow(epithelium_mask, cmap='gray')
#     axes[3].set_title("Epithelium Mask")
#     axes[3].axis('off')

#     # Show the plot
#     plt.tight_layout()
#     plt.show()

In [260]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")

Processing complete for all images.


### Eli's Code with Otsu's on Grayscale + Contour Selection stopping at 2 pieces

In [261]:
# Directories
input_folder = "Result TIF"
output_folder = "Eli + Otsu + 2 contours"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Thresholding to segment the epithelium
    _, thresholded_mask = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(thresholded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # Calculate areas for all contours and sort them in descending order
        contour_areas = [(cv2.contourArea(contour), contour) for contour in contours]
        contour_areas = sorted(contour_areas, key=lambda x: x[0], reverse=True)  # Sort based on area (first element of tuple)
    
        # Take the first 2 contours
        cutoff_index = 1

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(thresholded_mask)

        # Draw all contours up to the cutoff point
        selected_contours = [cont[1] for cont in contour_areas[:cutoff_index + 1]]
        cv2.drawContours(epithelium_mask, selected_contours, -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)
#         print(f"Average epithelium width for {image_path}: {average_epithelium_width}")

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

#         print(f"Processed and saved {output_image_path}, mask saved at {output_mask_path}")

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")
    
#     # Plotting multiple masks
#     fig, axes = plt.subplots(1, 4, figsize=(12, 6))

#     # First mask
#     axes[0].imshow(image, cmap='gray')
#     axes[0].set_title("Image")
#     axes[0].axis('off')
    
#     axes[1].imshow(blurred_image, cmap='gray')
#     axes[1].set_title("Blurred Image")
#     axes[1].axis('off')

#     # Second mask
#     axes[2].imshow(thresholded_mask, cmap='gray')
#     axes[2].set_title("Cleaned Mask")
#     axes[2].axis('off')
    
#     # Third mask
#     axes[3].imshow(epithelium_mask, cmap='gray')
#     axes[3].set_title("Epithelium Mask")
#     axes[3].axis('off')

#     # Show the plot
#     plt.tight_layout()
#     plt.show()

# process_image('Result TIF/h2114153 h&e.tif - Series 0_ROI_1.tif', 'Test/Image 1.tif', 'Test/Mask_Image 1.tif')

In [262]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")

Processing complete for all images.


### Eli's Code with Otsu's on Grayscale + Contour Selection stopping at 3 pieces

In [253]:
# Directories
input_folder = "Result TIF"
output_folder = "Eli + Otsu + 3 contours"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Thresholding to segment the epithelium
    _, thresholded_mask = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(thresholded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # Calculate areas for all contours and sort them in descending order
        contour_areas = [(cv2.contourArea(contour), contour) for contour in contours]
        contour_areas = sorted(contour_areas, key=lambda x: x[0], reverse=True)  # Sort based on area (first element of tuple)
    
        # Take the first 3 contours
        cutoff_index = 2

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(thresholded_mask)

        # Draw all contours up to the cutoff point
        selected_contours = [cont[1] for cont in contour_areas[:cutoff_index + 1]]
        cv2.drawContours(epithelium_mask, selected_contours, -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)
#         print(f"Average epithelium width for {image_path}: {average_epithelium_width}")

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

#         print(f"Processed and saved {output_image_path}, mask saved at {output_mask_path}")

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")
    
#     # Plotting multiple masks
#     fig, axes = plt.subplots(1, 4, figsize=(12, 6))

#     # First mask
#     axes[0].imshow(image, cmap='gray')
#     axes[0].set_title("Image")
#     axes[0].axis('off')
    
#     axes[1].imshow(blurred_image, cmap='gray')
#     axes[1].set_title("Blurred Image")
#     axes[1].axis('off')

#     # Second mask
#     axes[2].imshow(thresholded_mask, cmap='gray')
#     axes[2].set_title("Cleaned Mask")
#     axes[2].axis('off')
    
#     # Third mask
#     axes[3].imshow(epithelium_mask, cmap='gray')
#     axes[3].set_title("Epithelium Mask")
#     axes[3].axis('off')

#     # Show the plot
#     plt.tight_layout()
#     plt.show()

# process_image('Result TIF/h2114153 h&e.tif - Series 0_ROI_1.tif', 'Test/Image 1.tif', 'Test/Mask_Image 1.tif')

In [254]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")

Processing complete for all images.


### Eli's Code with Otsu's on Grayscale + Contour Selection stopping at 5 pieces

In [263]:
# Directories
input_folder = "Result TIF"
output_folder = "Eli + Otsu + 5 contours"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Thresholding to segment the epithelium
    _, thresholded_mask = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(thresholded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # Calculate areas for all contours and sort them in descending order
        contour_areas = [(cv2.contourArea(contour), contour) for contour in contours]
        contour_areas = sorted(contour_areas, key=lambda x: x[0], reverse=True)  # Sort based on area (first element of tuple)

        # Take the first 5 contours
        cutoff_index = 4

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(thresholded_mask)

        # Draw all contours up to the cutoff point
        selected_contours = [cont[1] for cont in contour_areas[:cutoff_index + 1]]
        cv2.drawContours(epithelium_mask, selected_contours, -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)
#         print(f"Average epithelium width for {image_path}: {average_epithelium_width}")

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

#         print(f"Processed and saved {output_image_path}, mask saved at {output_mask_path}")

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")
    
#     # Plotting multiple masks
#     fig, axes = plt.subplots(1, 4, figsize=(12, 6))

#     # First mask
#     axes[0].imshow(image, cmap='gray')
#     axes[0].set_title("Image")
#     axes[0].axis('off')
    
#     axes[1].imshow(blurred_image, cmap='gray')
#     axes[1].set_title("Blurred Image")
#     axes[1].axis('off')

#     # Second mask
#     axes[2].imshow(thresholded_mask, cmap='gray')
#     axes[2].set_title("Cleaned Mask")
#     axes[2].axis('off')
    
#     # Third mask
#     axes[3].imshow(epithelium_mask, cmap='gray')
#     axes[3].set_title("Epithelium Mask")
#     axes[3].axis('off')

#     # Show the plot
#     plt.tight_layout()
#     plt.show()

# process_image('Result TIF/h2114153 h&e.tif - Series 0_ROI_1.tif', 'Test/Image 1.tif', 'Test/Mask_Image 1.tif')

In [264]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")

Processing complete for all images.


### Eli's Code with Otsu's and LAB images

In [247]:
# Directories
input_folder = "Result TIF"
output_folder = "Eli + Otsu + LAB"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")
        
    # Convert to LAB color space
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Extract the L channel
    l_channel = image_lab[:,:,0]

    # Apply Otsu's thresholding
    _, otsu_threshold = cv2.threshold(l_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Create a mask using the Otsu threshold
    mask = np.zeros(l_channel.shape, dtype=np.uint8)
    mask[l_channel > otsu_threshold] = 255
    
    # Ensure mask is the same size as the image
    if mask.shape[:2] != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(mask)
        cv2.drawContours(epithelium_mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)
#         print(f"Average epithelium width for {image_path}: {average_epithelium_width}")

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

#         print(f"Processed and saved {output_image_path}, mask saved at {output_mask_path}")

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")
    
#     # Plotting multiple masks
#     fig, axes = plt.subplots(1, 4, figsize=(12, 6))

#     # First mask
#     axes[0].imshow(image, cmap='gray')
#     axes[0].set_title("Image")
#     axes[0].axis('off')
    
#     axes[1].imshow(blurred_image, cmap='gray')
#     axes[1].set_title("Blurred Image")
#     axes[1].axis('off')

#     # Second mask
#     axes[2].imshow(mask, cmap='gray')
#     axes[2].set_title("Cleaned Mask")
#     axes[2].axis('off')
    
#     # Third mask
#     axes[3].imshow(epithelium_mask, cmap='gray')
#     axes[3].set_title("Epithelium Mask")
#     axes[3].axis('off')

#     # Show the plot
#     plt.tight_layout()
#     plt.show()

In [ ]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")

### Eli's Code with Otsu's and LAB images + Contour Selection, stopping at 10%

In [249]:
# Directories
input_folder = "Result TIF"
output_folder = "Eli + Otsu + LAB + 10%"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")
        
    # Convert to LAB color space
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Extract the L channel
    l_channel = image_lab[:,:,0]

    # Apply Otsu's thresholding
    _, otsu_threshold = cv2.threshold(l_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Create a mask using the Otsu threshold
    mask = np.zeros(l_channel.shape, dtype=np.uint8)
    mask[l_channel > otsu_threshold] = 255
    
    # Ensure mask is the same size as the image
    if mask.shape[:2] != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # Calculate areas for all contours and sort them in descending order
        contour_areas = [(cv2.contourArea(contour), contour) for contour in contours]
        contour_areas = sorted(contour_areas, key=lambda x: x[0], reverse=True)  # Sort based on area (first element of tuple)
        
#         print("Contour areas (sorted from largest to smallest):")
#         for i, (area, _) in enumerate(contour_areas):
#             print(f"Contour {i}: {area} pixels")
        
#         largest_contour = max(contours, key=cv2.contourArea)

        # Calculate the percentage difference between consecutive areas
        area_differences = []
        for i in range(len(contour_areas)-1):
            current_area = contour_areas[i][0]
            next_area = contour_areas[i+1][0]
            if current_area > 0:  # Avoid division by zero
                percentage = (next_area / current_area) * 100
                area_differences.append((percentage, i))
    
        # Find the index where there's a significant drop (e.g., >50% difference)
        cutoff_index = 0  # Default to just the largest contour
        for diff, idx in area_differences:
            if diff < 10:  # You can adjust this threshold
                cutoff_index = idx
                break

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(mask)

        # Draw all contours up to the cutoff point
        selected_contours = [cont[1] for cont in contour_areas[:cutoff_index + 1]]
        cv2.drawContours(epithelium_mask, selected_contours, -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)
#         print(f"Average epithelium width for {image_path}: {average_epithelium_width}")

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

#         print(f"Processed and saved {output_image_path}, mask saved at {output_mask_path}")

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")

In [ ]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")

### Eli's Code with Otsu's and LAB images + Contour Selection, stopping at 2 contours

In [250]:
# Directories
input_folder = "Result TIF"
output_folder = "Eli + Otsu + LAB + 2 contours"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")
        
    # Convert to LAB color space
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Extract the L channel
    l_channel = image_lab[:,:,0]

    # Apply Otsu's thresholding
    _, otsu_threshold = cv2.threshold(l_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Create a mask using the Otsu threshold
    mask = np.zeros(l_channel.shape, dtype=np.uint8)
    mask[l_channel > otsu_threshold] = 255
    
    # Ensure mask is the same size as the image
    if mask.shape[:2] != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # Calculate areas for all contours and sort them in descending order
        contour_areas = [(cv2.contourArea(contour), contour) for contour in contours]
        contour_areas = sorted(contour_areas, key=lambda x: x[0], reverse=True)  # Sort based on area (first element of tuple)
    
        # Take the first 2 indicies
        cutoff_index = 1  # Default to just the largest contour

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(mask)

        # Draw all contours up to the cutoff point
        selected_contours = [cont[1] for cont in contour_areas[:cutoff_index + 1]]
        cv2.drawContours(epithelium_mask, selected_contours, -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)
#         print(f"Average epithelium width for {image_path}: {average_epithelium_width}")

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

#         print(f"Processed and saved {output_image_path}, mask saved at {output_mask_path}")

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")
    
#     # Plotting multiple masks
#     fig, axes = plt.subplots(1, 4, figsize=(12, 6))

#     # First mask
#     axes[0].imshow(image, cmap='gray')
#     axes[0].set_title("Image")
#     axes[0].axis('off')
    
#     axes[1].imshow(blurred_image, cmap='gray')
#     axes[1].set_title("Blurred Image")
#     axes[1].axis('off')

#     # Second mask
#     axes[2].imshow(mask, cmap='gray')
#     axes[2].set_title("Cleaned Mask")
#     axes[2].axis('off')
    
#     # Third mask
#     axes[3].imshow(epithelium_mask, cmap='gray')
#     axes[3].set_title("Epithelium Mask")
#     axes[3].axis('off')

#     # Show the plot
#     plt.tight_layout()
#     plt.show()

# process_image('Result TIF/h2114153 h&e.tif - Series 0_ROI_1.tif', 'Test/Image 1.tif', 'Test/Mask_Image 1.tif')

In [ ]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")

### Eli's Code with Otsu's and LAB images + Contour Selection, stopping at 3 contours

In [251]:
# Directories
input_folder = "Result TIF"
output_folder = "Eli + Otsu + LAB + 3 contours"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")
        
    # Convert to LAB color space
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Extract the L channel
    l_channel = image_lab[:,:,0]

    # Apply Otsu's thresholding
    _, otsu_threshold = cv2.threshold(l_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Create a mask using the Otsu threshold
    mask = np.zeros(l_channel.shape, dtype=np.uint8)
    mask[l_channel > otsu_threshold] = 255
    
    # Ensure mask is the same size as the image
    if mask.shape[:2] != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # Calculate areas for all contours and sort them in descending order
        contour_areas = [(cv2.contourArea(contour), contour) for contour in contours]
        contour_areas = sorted(contour_areas, key=lambda x: x[0], reverse=True)  # Sort based on area (first element of tuple)
    
        # Take the first 3 indicies
        cutoff_index = 2  # Default to just the largest contour

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(mask)

        # Draw all contours up to the cutoff point
        selected_contours = [cont[1] for cont in contour_areas[:cutoff_index + 1]]
        cv2.drawContours(epithelium_mask, selected_contours, -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)
#         print(f"Average epithelium width for {image_path}: {average_epithelium_width}")

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

#         print(f"Processed and saved {output_image_path}, mask saved at {output_mask_path}")

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")
    
#     # Plotting multiple masks
#     fig, axes = plt.subplots(1, 4, figsize=(12, 6))

#     # First mask
#     axes[0].imshow(image, cmap='gray')
#     axes[0].set_title("Image")
#     axes[0].axis('off')
    
#     axes[1].imshow(blurred_image, cmap='gray')
#     axes[1].set_title("Blurred Image")
#     axes[1].axis('off')

#     # Second mask
#     axes[2].imshow(mask, cmap='gray')
#     axes[2].set_title("Cleaned Mask")
#     axes[2].axis('off')
    
#     # Third mask
#     axes[3].imshow(epithelium_mask, cmap='gray')
#     axes[3].set_title("Epithelium Mask")
#     axes[3].axis('off')

#     # Show the plot
#     plt.tight_layout()
#     plt.show()

# process_image('Result TIF/h2114153 h&e.tif - Series 0_ROI_1.tif', 'Test/Image 1.tif', 'Test/Mask_Image 1.tif')

In [ ]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")

### Eli's Code with Otsu's and LAB images + Contour Selection, stopping at 5 contours

In [252]:
# Directories
input_folder = "Result TIF"
output_folder = "Eli + Otsu + LAB + 5 contours"

# Check if output folder exists, if not create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to process each image
def process_image(image_path, output_image_path, output_mask_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")
        
    # Convert to LAB color space
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Extract the L channel
    l_channel = image_lab[:,:,0]

    # Apply Otsu's thresholding
    _, otsu_threshold = cv2.threshold(l_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Create a mask using the Otsu threshold
    mask = np.zeros(l_channel.shape, dtype=np.uint8)
    mask[l_channel > otsu_threshold] = 255
    
    # Ensure mask is the same size as the image
    if mask.shape[:2] != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Find contours to identify the epithelium region
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # Calculate areas for all contours and sort them in descending order
        contour_areas = [(cv2.contourArea(contour), contour) for contour in contours]
        contour_areas = sorted(contour_areas, key=lambda x: x[0], reverse=True)  # Sort based on area (first element of tuple)
    
        # Take the first 5 indicies
        cutoff_index = 4  # Default to just the largest contour

        # Create a blank mask to draw the epithelium contour
        epithelium_mask = np.zeros_like(mask)

        # Draw all contours up to the cutoff point
        selected_contours = [cont[1] for cont in contour_areas[:cutoff_index + 1]]
        cv2.drawContours(epithelium_mask, selected_contours, -1, (255), thickness=cv2.FILLED)

        # Save the mask image
        cv2.imwrite(output_mask_path, epithelium_mask)

        # Calculate the width of the epithelium at multiple segments for each row
        def calculate_average_epithelium_width(mask):
            height = mask.shape[0]
            widths = []

            # Iterate through each row (y) in the image
            for y in range(height):
                row = mask[y, :]  # Get the row
                white_pixel_indices = np.where(row == 255)[0]  # Get the indices of white pixels (epithelium)

                # If there are white pixels in the row
                if len(white_pixel_indices) > 0:
                    # Separate the contiguous white segments
                    segments = np.split(white_pixel_indices, np.where(np.diff(white_pixel_indices) != 1)[0] + 1)

                    # Calculate the width of each segment and add to widths list
                    for segment in segments:
                        if len(segment) > 0:
                            width = segment[-1] - segment[0]
                            widths.append(width)

            # Calculate and return the average of all widths
            if len(widths) > 0:
                average_width = np.mean(widths)
            else:
                average_width = 0

            return int(average_width)

        # Get the average width of the epithelium
        average_epithelium_width = calculate_average_epithelium_width(epithelium_mask)
#         print(f"Average epithelium width for {image_path}: {average_epithelium_width}")

        # Define the sliding window function
        def sliding_window(image, mask, step_size, window_size):
            # Slide a window across the image
            for y in range(0, image.shape[0] - window_size, step_size):
                for x in range(0, image.shape[1] - window_size, step_size):
                    yield (x, y, image[y:y + window_size, x:x + window_size], mask[y:y + window_size, x:x + window_size])

        # Limit the box size to 20% of the image size
        max_box_size = min(image.shape[0], image.shape[1]) * 0.2

        # Now applying the sliding window logic based on the mask
        window_size = min(average_epithelium_width, int(max_box_size))  # Square box size based on average epithelium width
        stride = window_size  # No overlap

        final_windows_image = image.copy()

        # Sliding window process
        for (x, y, window, window_mask) in sliding_window(image, epithelium_mask, stride, window_size):
            
            # Calculate the ratio of epithelium (white) area in the window
            total_area = window_mask.size
            epithelium_area = np.count_nonzero(window_mask)
            epithelium_ratio = epithelium_area / total_area
            
            # Draw a red box around the window if epithelium occupies more than 50%
            if epithelium_ratio > 0.50:
                # Expand the box
                x_start, y_start = x, y
                x_end, y_end = x + window_size, y + window_size

                # Ensure box does not exceed the max size
                if (x_end - x_start) <= max_box_size and (y_end - y_start) <= max_box_size:
                    cv2.rectangle(final_windows_image, (x_start, y_start), (x_end, y_end), (0, 0, 255), 8)

        # Save the final image with the windows drawn
        cv2.imwrite(output_image_path, final_windows_image)

#         print(f"Processed and saved {output_image_path}, mask saved at {output_mask_path}")

    else:
        print(f"No epithelium contours were detected in {image_path}. Try adjusting the parameters.")
    
#     # Plotting multiple masks
#     fig, axes = plt.subplots(1, 4, figsize=(12, 6))

#     # First mask
#     axes[0].imshow(image, cmap='gray')
#     axes[0].set_title("Image")
#     axes[0].axis('off')
    
#     axes[1].imshow(blurred_image, cmap='gray')
#     axes[1].set_title("Blurred Image")
#     axes[1].axis('off')

#     # Second mask
#     axes[2].imshow(mask, cmap='gray')
#     axes[2].set_title("Cleaned Mask")
#     axes[2].axis('off')
    
#     # Third mask
#     axes[3].imshow(epithelium_mask, cmap='gray')
#     axes[3].set_title("Epithelium Mask")
#     axes[3].axis('off')

#     # Show the plot
#     plt.tight_layout()
#     plt.show()

# process_image('Result TIF/h2114153 h&e.tif - Series 0_ROI_1.tif', 'Test/Image 1.tif', 'Test/Mask_Image 1.tif')

In [ ]:
# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust based on your file types
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, f"filtered_{filename}")
        output_mask_path = os.path.join(output_folder, f"mask_{filename}")

        # Process the image and save the output and mask
        process_image(input_image_path, output_image_path, output_mask_path)

print("Processing complete for all images.")